In [1]:
# instalacion de openpyxl

import openpyxl
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3 as sql3
from matplotlib import rcParams

In [2]:
# Importacion de articles.db (sql) de la carpeta data y trannsformacion a dataframe
conn = sql3.connect('data/articles.db')

# se toma la columna article_id como index del dataframe
# y como el metodo read_sql_query no la elimina,
# hago un drop de la columna posterior, ya que todos NaN y no 
 # tiene opcion inplace.

sql_query = pd.read_sql_query("SELECT * FROM articles", conn, index_col='article_id')
df_articles = pd.DataFrame(sql_query, columns=['article_id', 'article_name','unit_price'])
df_articles = df_articles.drop(columns='article_id', axis=1)
# imprimo el dataframe
print(df_articles.head(2))

           article_name unit_price
article_id                        
20015        Smartphone     525.00
20016           Full Pc    2127.81


In [3]:
# importacio de sales.db (sql) de la carpeta data y transformacion a dataframe
df_sellers = pd.read_excel('data/sellers.xlsx', index_col='seller_id')
# imprimo el dataframe
print(df_sellers.head(4))


                 seller_name
seller_id                   
1           Aveline Swanwick
2                   Jase Doy
3          Oliviero Charkham
4           Cornie Wynrehame


In [4]:
# importacion de orders.csv de la carpeta data y transformacion a dataframe
df_orders = pd.read_csv('data/orders.csv')
# imprimo el dataframe
print(df_orders.head(3))

   order_id  week  article_id  quantity  seller_id country_name
0     15024     1       20039        10         10         Peru
1     15025     1       20029        15          5         Peru
2     15026     1       20024         5         14      Bolivia


In [5]:
# explorar y preparar datos
# exploracion del dataframe df_articles
print("Muestra de datos")
print(df_articles.sample())

Muestra de datos
           article_name unit_price
article_id                        
20016           Full Pc    2127.81


In [6]:
print("Dimensiones del dataframe")
print(df_articles.shape)

Dimensiones del dataframe
(31, 2)


In [7]:
print("Busqueda de valores nulos")
print(df_articles.isnull().sum())

Busqueda de valores nulos
article_name    0
unit_price      0
dtype: int64


In [8]:
print("Tipo de datos por columna")
print(df_articles.dtypes)

Tipo de datos por columna
article_name    object
unit_price      object
dtype: object


In [9]:
# exploracion del dataframe df_sellers
print("Muestra de datos")
print(df_sellers.sample())

Muestra de datos
               seller_name
seller_id                 
13         Arnold Kilkenny


In [10]:
print("Dimensiones del dataframe")
print(df_sellers.shape)

Dimensiones del dataframe
(15, 1)


In [11]:
print("Busqueda de valores nulos")
print(df_sellers.isnull().sum())

Busqueda de valores nulos
seller_name    0
dtype: int64


In [12]:
print("Tipo de datos por columna")
print(df_sellers.dtypes)

Tipo de datos por columna
seller_name    object
dtype: object


In [13]:
# exploracion del dataframe df_orders
print("Muestra de datos")
print(df_orders.sample())

Muestra de datos
     order_id  week  article_id  quantity  seller_id country_name
468     15492     2       20017        14         15   Costa Rica


In [14]:
print("Dimensiones del dataframe")
print(df_orders.shape)

Dimensiones del dataframe
(1000, 6)


In [15]:
print("Busqueda de valores nulos")
print(df_orders.isnull().sum())


Busqueda de valores nulos
order_id        0
week            0
article_id      0
quantity        0
seller_id       0
country_name    0
dtype: int64


In [16]:
print("Tipo de datos por columna")
print(df_orders.dtypes)

Tipo de datos por columna
order_id         int64
week             int64
article_id       int64
quantity         int64
seller_id        int64
country_name    object
dtype: object


In [17]:
# cambiar el formato de la columna precios de los articulos
# de string a float
pd.options.display.float_format = '$ {:,.2f}'.format
df_articles['unit_price'] = df_articles['unit_price'].astype(float)
# imprimo el dataframe
print(df_articles.dtypes)

article_name     object
unit_price      float64
dtype: object


In [18]:
print(df_articles.head(3))

           article_name  unit_price
article_id                         
20015        Smartphone    $ 525.00
20016           Full Pc  $ 2,127.81
20017           Monitor    $ 230.00


<h2>Analisis de datos</h2>
<h3>Definir el modelo de datos</h3>
<img src="img/Diagrama.png" alt="Modelo de datos" style="width: 600px;"/>

In [19]:
data = df_orders.copy()
data.head(2)

,order_id,week,article_id,quantity,seller_id,country_name
0,15024,1,20039,10,10,Peru
1,15025,1,20029,15,5,Peru


In [20]:
# Voy a unir los distintos DataFrame utilizando el método merge (equivalente en Pandas a JOIN en SQL) uniendo de la siguiente manera:
# article_id del DataFrame Data con el index de df_articles para obtener el nombre y precio unitario
# seller_id del DataFrame Data con el index de df_sellers para obtener el nombre del vendedor
# Posteriormente tengo dropear las columnas article_id y seller_id del DataFrame data ya que no tiene sentido que sigan ahí.

data_article = data.merge(df_articles, left_on='article_id', right_on=df_articles.index, how='inner')
data_article.head(3)

,order_id,week,article_id,quantity,seller_id,country_name,article_name,unit_price
0,15024,1,20039,10,10,Peru,Water Cooling,$ 67.50
1,15047,1,20039,5,9,Argentina,Water Cooling,$ 67.50
2,15058,1,20039,9,12,Mexico,Water Cooling,$ 67.50


In [21]:
data_article_seller = data_article.merge(df_sellers, left_on='seller_id', right_on=df_sellers.index, how='inner')
data_article_seller.head(3)

,order_id,week,article_id,quantity,seller_id,country_name,article_name,unit_price,seller_name
0,15024,1,20039,10,10,Peru,Water Cooling,$ 67.50,Cirilo Grandham
1,15892,3,20039,15,10,Brazil,Water Cooling,$ 67.50,Cirilo Grandham
2,15732,3,20029,6,10,Guatemala,Mouse,$ 30.30,Cirilo Grandham


In [22]:
data_unificada = data_article_seller.drop(columns=['article_id', 'seller_id'], axis=1)
data_unificada.head(3)

,order_id,week,quantity,country_name,article_name,unit_price,seller_name
0,15024,1,10,Peru,Water Cooling,$ 67.50,Cirilo Grandham
1,15892,3,15,Brazil,Water Cooling,$ 67.50,Cirilo Grandham
2,15732,3,6,Guatemala,Mouse,$ 30.30,Cirilo Grandham


In [23]:
# crear nueva columna con el total de la venta y eliminar las columnas unit_price
data_unificada['total_amount'] = data_unificada['unit_price'] * data_unificada['quantity']
data_unificada.head(3)

,order_id,week,quantity,country_name,article_name,unit_price,seller_name,total_amount
0,15024,1,10,Peru,Water Cooling,$ 67.50,Cirilo Grandham,$ 675.00
1,15892,3,15,Brazil,Water Cooling,$ 67.50,Cirilo Grandham,"$ 1,012.50"
2,15732,3,6,Guatemala,Mouse,$ 30.30,Cirilo Grandham,$ 181.80


In [24]:
data_a_trabajar = data_unificada.drop(['unit_price', 'order_id'], axis=1)
data_a_trabajar.head(3)

,week,quantity,country_name,article_name,seller_name,total_amount
0,1,10,Peru,Water Cooling,Cirilo Grandham,$ 675.00
1,3,15,Brazil,Water Cooling,Cirilo Grandham,"$ 1,012.50"
2,3,6,Guatemala,Mouse,Cirilo Grandham,$ 181.80


In [25]:
data_a_trabajar.dtypes

week              int64
quantity          int64
country_name     object
article_name     object
seller_name      object
total_amount    float64
dtype: object